In [ ]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt


import holodeck as holo
from holodeck import detstats, utils, sam
import holodeck.detstats as ds
from holodeck.constants import YR, MSOL

import hasasia.sensitivity as hsen
import hasasia.sim as hsim 
import hasasia.skymap as hsky

# 1 Strain Model

Build SAM and calculate strains

Make the semi-analytic model object.

In [ ]:
dur = 10.0*YR
cad = 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
# sam = holo.sam.Semi_Analytic_Model() 
sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version
hard = holo.hardening.Hard_GW()

Calculate strains.

In [ ]:
fobs=fobs_gw_cents
dfobs = np.diff(fobs_gw_edges)

hc_ss, hc_bg = sam.gwb(fobs_gw_edges, hard, realize=30, 
                                        loudest = 5, params = False) 
print(hc_ss.shape)

# 1.) 2 pulsars, 1 binary
## 1.1 Setup
### 1.1.1 Build SAM and calculate strains

Make the semi-analytic model object.

In [ ]:
# hcfile = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/sample_sam_C.npz')
# dur = hcfile['dur']
# cad = hcfile['cad']
# fobs = hcfile['fobs']
# dfobs = hcfile['dfobs']
# hc_ss = hcfile['hc_ss']
# hc_bg = hcfile['hc_bg']
# print(hc_ss.shape)

### 1.1.2 Set Up 1 SS


Place the single sources randomly across the sky
Isolate the 3rd frequency bin source, place it.

In [ ]:
theta_ss = np.random.uniform(0, np.pi, size = hc_ss.size).reshape(hc_ss.shape)
phi_ss = np.random.uniform(0, 2*np.pi, size = hc_ss.size).reshape(hc_ss.shape)
iota_ss = np.random.uniform(0, np.pi, size = hc_ss.size).reshape(hc_ss.shape)
psi_ss = np.random.uniform(0, np.pi, size = hc_ss.size).reshape(hc_ss.shape)
Phi0_ss = np.random.uniform(0,2*np.pi, size=hc_ss.size).reshape(hc_ss.shape)
# np.savez('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/random_binary_angles_C.npz',
#          theta_ss=theta_ss, phi_ss=phi_ss, iota_ss=iota_ss, psi_ss=psi_ss, Phi0_ss=Phi0_ss)

# infile = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/detstats_functions/npz_arrays/random_binary_angles_C.npz')
# theta_ss = infile['theta_ss']
# phi_ss = infile['phi_ss']
# iota_ss = infile['iota_ss']
# psi_ss = infile['psi_ss']
# Phi0_ss = infile['Phi0_ss']

fig = holo.plot.plot_bg_ss(fobs, hc_bg, hc_ss)

### 1.1.3 Set Up 2 Pulsars
* 2 pulsars
* 45 degrees ($\pi$/4) apart in $\phi$
* same $\theta=0$

In [ ]:
# set pulsar parameters
def reset_pulsars(npsrs=2, phis = np.array([0,np.pi/4]), thetas = np.array([0,0]), 
    sigmas = np.array([1e-7,1e-7])):
    # build sim_pta
    pulsars = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                        phi=phis, theta=thetas)
    # get spectrum for each
    spectra = np.empty_like(pulsars, dtype=hsen.Spectrum)
    for ii in range(npsrs):
        spectra[ii] = hsen.Spectrum(pulsars[ii], freqs=fobs)
        spectra[ii].NcalInv # calculate inverse noise weighted transmission function
    # get sensitivity curve
    # scDeter = hsen.DeterSensitivityCurve(list(spectra))
    # skymap = hsky.SkySensitivity(list(spectra), theta_gw=theta_ss.squeeze(), 
    #                              phi_gw=phi_ss.squeeze(),
    #                              iota=iota_ss.squeeze())
    return npsrs, thetas, phis, sigmas, pulsars, spectra #, scDeter, skymap
npsrs, thetas, phis, sigmas, pulsars, spectra = reset_pulsars()

## 1.2 SNR 

In [ ]:
# amp = detstats._amplitude(hc_ss, fobs, dfobs)
# F_iplus, F_icross = detstats._antenna_pattern_functions(m_hat, n_hat, Omega_hat, pi_hat)

# snr_ss_cython = detstats._snr_ss(amp, F_iplus, F_icross, iotas, dur, Phi_0, S_i, freqs=fobs)

In [ ]:
num = (hc_ss[0]).size
print(num)

In [ ]:
dp_ss, snr_ss, gamma_ssi = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg, 
                       theta_ss, phi_ss, iota_ss, Phi0_ss, Fe_bar_guess=25, ret_snr=True)

In [ ]:
 # unitary vectors
m_hat = ds._m_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
print(m_hat.shape)
n_hat = ds._n_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
Omega_hat = ds._Omega_unitary_vector(theta_ss, phi_ss) # (3,F,S,L)
pi_hat = ds._pi_unitary_vector(phis, thetas) # (3,P)

# antenna pattern functions
F_iplus, F_icross = ds._antenna_pattern_functions(m_hat, n_hat, Omega_hat, 
                                                pi_hat) # (P,F,S,L)
print(F_iplus.shape)

# noise spectral density
S_i = ds._total_noise(cad, sigmas, hc_ss, hc_bg, fobs)

# amplitudw
amp = ds._amplitude(hc_ss, fobs, 
                    dfobs) # (F,R,L)
print(amp.shape)

snr_ss_5d = ds._snr_ss_5dim(amp, F_iplus, F_icross, iota_ss, dur, Phi0_ss, S_i, fobs) # (F,R,S,L)

In [ ]:
plt.scatter(snr_ss.flatten(), snr_ss_5d.flatten(), s=1, alpha=0.5)

In [ ]:
print(hc_ss.shape, theta_ss.shape)

### 2.1.3 Place 40 random pulsars

In [ ]:
npsrs = 40
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
sigmas = np.ones_like(phis)*1e-7

# build sim_pta
pulsars = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigmas,
                    phi=phis, theta=thetas)
# print(theta_ss.squeeze(), '\n', phi_ss.squeeze(), '\n', iota_ss.squeeze())

In [ ]:
dp_ss, snr_ss, gamma_ssi = detstats.detect_ss_pta(pulsars, cad, dur, fobs, dfobs, hc_ss, hc_bg, 
                       theta_ss, phi_ss, iota_ss, Phi0_ss, Fe_bar_guess=25, ret_snr=True)

In [ ]:
 # unitary vectors
m_hat = ds._m_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
print(m_hat.shape)
n_hat = ds._n_unitary_vector(theta_ss, phi_ss, psi_ss) # (3,F,S,L)
Omega_hat = ds._Omega_unitary_vector(theta_ss, phi_ss) # (3,F,S,L)
pi_hat = ds._pi_unitary_vector(phis, thetas) # (3,P)

# antenna pattern functions
F_iplus, F_icross = ds._antenna_pattern_functions(m_hat, n_hat, Omega_hat, 
                                                pi_hat) # (P,F,S,L)
print(F_iplus.shape)

# noise spectral density
S_i = ds._total_noise(cad, sigmas, hc_ss, hc_bg, fobs)

# amplitudw
amp = ds._amplitude(hc_ss, fobs, 
                    dfobs) # (F,R,L)
print(amp.shape)

snr_ss_5d = ds._snr_ss_5dim(amp, F_iplus, F_icross, iota_ss, dur, Phi0_ss, S_i, fobs) # (F,R,S,L)

In [ ]:
fig, ax = plot.figax(xlabel='snr_ss with cython', ylabel='snr_ss with 5darrays')
ax.scatter(snr_ss.flatten(), snr_ss_5d.flatten(), s=1, alpha=0.5)
ax.plot(np.arange(50), np.arange(50), color='k', alpha=0.5)

In [ ]:
fig, ax = plot.figax(xlabel='snr_ss with cython', ylabel='snr_ss with 5darrays', yscale='linear', xscale='linear')
ax.scatter(snr_ss.flatten(), snr_ss_5d.flatten(), s=1, alpha=0.5)
ax.plot(np.arange(50), np.arange(50), color='k', alpha=0.5)